### Imports

In [16]:
import pandas as pd
import os
import time
import numpy as np
import torch
from datasets import load_dataset 
from CodeSyntaxConcept.tokenizer import CodeTokenizer
import CodeSyntaxConcept.utils as utils
from statistics import mean, median
import json

### Parameters

In [17]:
checkpoint = "EleutherAI/gpt-neo-125M"
file_path = "output/testbed_base_EleutherAI-gpt-neo-125M.csv"
parent_node_types_path = "output/parent_node_types.csv"
child_node_types_path = "output/child_node_types.csv"
language = "python"
aggregates_path = "output/aggregation_function/" + file_path.split('/')[1]

In [18]:
tokenizer = CodeTokenizer.from_pretrained(checkpoint, language)

### Parent Node Types

In [4]:
parent_node_types = pd.read_csv(parent_node_types_path, index_col=0)
parent_node_types = set(parent_node_types['parent_type'])

### Children Node Types

In [5]:
child_node_types = pd.read_csv(child_node_types_path, index_col=0)
child_node_types = set(child_node_types['child_type'])

### Load Aggregates

In [6]:
df_actual_ntp = pd.read_csv(aggregates_path, index_col=0)

In [7]:
##Convert JSON do Dict
df_actual_ntp['binded_tree'] = df_actual_ntp['binded_tree'].map(lambda binded_tree: json.loads(binded_tree))

In [8]:
df_actual_ntp.head()

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids,max_prob_case,min_prob_case,actual_prob_case,binded_tree
0,"def get_node(self, label):\n """"""\n ...","[('def', 'def', 'function_definition'), ('get_...","[(4299, 'def', 'function_definition'), (651, '...","[4299, 651, 62, 17440, 7, 944, 11, 6167, 2599,...",115,"[('ers', 0.03083181567490101), ('_', 0.2419087...","[('anwhile', 1.03836617568492e-16), ('ousy', 3...","[('Ġget', 0.0024285861290991306), ('_', 0.2419...","{'type': 'module', 'children': [{'type': 'func..."
1,"def execute_pipeline(pipeline, environment_dic...","[('def', 'def', 'function_definition'), ('exec...","[(4299, 'def', 'function_definition'), (12260,...","[4299, 12260, 62, 79, 541, 4470, 7, 79, 541, 4...",492,"[('ers', 0.03083181567490101), ('_', 0.1853068...","[('anwhile', 1.03836617568492e-16), ('icester'...","[('Ġexecute', 5.97450380155351e-05), ('_', 0.1...","{'type': 'module', 'children': [{'type': 'func..."
2,"def _decode(self, data):\n '''\n ...","[('def', 'def', 'function_definition'), ('_dec...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 12501, 1098, 7, 944, 11, 1366, 25...",583,"[('ers', 0.03083181567490101), ('(', 0.0166473...","[('anwhile', 1.03836617568492e-16), ('ousy', 4...","[('Ġ_', 0.001834857277572155), ('dec', 0.00133...","{'type': 'module', 'children': [{'type': 'func..."
3,"def _repr_html_(self):\n """"""\n J...","[('def', 'def', 'function_definition'), ('_rep...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 260, 1050, 62, 6494, 41052, 944, ...",221,"[('ers', 0.03083181567490101), ('(', 0.0166473...","[('anwhile', 1.03836617568492e-16), ('ousy', 4...","[('Ġ_', 0.001834857277572155), ('re', 0.001378...","{'type': 'module', 'children': [{'type': 'func..."
4,"def build_shape(relation, nodes, ways):\n ""...","[('def', 'def', 'function_definition'), ('buil...","[(4299, 'def', 'function_definition'), (1382, ...","[4299, 1382, 62, 43358, 7, 49501, 11, 13760, 1...",454,"[('ers', 0.03083181567490101), ('_', 0.2842166...","[('anwhile', 1.03836617568492e-16), ('buquerqu...","[('Ġbuild', 0.0005856865900568664), ('_', 0.28...","{'type': 'module', 'children': [{'type': 'func..."


### Local Analysis (Snippets)

In [9]:
def traverse_tree_and_collect_stds(node: dict, node_types_list: list, std_field: str):
    if node[std_field] is not None:
        node_types_list[tokenizer.node_types.index(node['type'])] = node_types_list[tokenizer.node_types.index(node['type'])] + [node[std_field]]
    for child in node['children']:
        traverse_tree_and_collect_stds(child, node_types_list, std_field)

In [10]:
def add_statistic_column(std_field, dataframe):
    concept_probs = []
    for tree in dataframe['binded_tree']:
        node_types_list = [[] for type in tokenizer.node_types]
        traverse_tree_and_collect_stds(tree, node_types_list, std_field)
        snippet_type_list = []
        for type_index, node_values in enumerate(node_types_list):
            if len(node_values)>0: 
                snippet_type_list.append((tokenizer.node_types[type_index], node_values))
        concept_probs.append(snippet_type_list)
    dataframe['concept_'+std_field] =  concept_probs

In [11]:
add_statistic_column('median_prob', df_actual_ntp)
add_statistic_column('min_prob', df_actual_ntp)
add_statistic_column('max_prob', df_actual_ntp)
df_actual_ntp.head()

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids,max_prob_case,min_prob_case,actual_prob_case,binded_tree,concept_median_prob,concept_min_prob,concept_max_prob
0,"def get_node(self, label):\n """"""\n ...","[('def', 'def', 'function_definition'), ('get_...","[(4299, 'def', 'function_definition'), (651, '...","[4299, 651, 62, 17440, 7, 944, 11, 6167, 2599,...",115,"[('ers', 0.03083181567490101), ('_', 0.2419087...","[('anwhile', 1.03836617568492e-16), ('ousy', 3...","[('Ġget', 0.0024285861290991306), ('_', 0.2419...","{'type': 'module', 'children': [{'type': 'func...","[(:, [0.08046948164701462, 0.01487241126596927...","[(:, [0.08046948164701462, 0.01487241126596927...","[(:, [0.08046948164701462, 0.01487241126596927..."
1,"def execute_pipeline(pipeline, environment_dic...","[('def', 'def', 'function_definition'), ('exec...","[(4299, 'def', 'function_definition'), (12260,...","[4299, 12260, 62, 79, 541, 4470, 7, 79, 541, 4...",492,"[('ers', 0.03083181567490101), ('_', 0.1853068...","[('anwhile', 1.03836617568492e-16), ('icester'...","[('Ġexecute', 5.97450380155351e-05), ('_', 0.1...","{'type': 'module', 'children': [{'type': 'func...","[(as_pattern_target, [0.22679175436496735]), (...","[(as_pattern_target, [0.0003977296582888812]),...","[(as_pattern_target, [0.790716826915741]), (:,..."
2,"def _decode(self, data):\n '''\n ...","[('def', 'def', 'function_definition'), ('_dec...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 12501, 1098, 7, 944, 11, 1366, 25...",583,"[('ers', 0.03083181567490101), ('(', 0.0166473...","[('anwhile', 1.03836617568492e-16), ('ousy', 4...","[('Ġ_', 0.001834857277572155), ('dec', 0.00133...","{'type': 'module', 'children': [{'type': 'func...","[(:, [0.26907914876937866, 0.530799925327301, ...","[(:, [0.26907914876937866, 0.530799925327301, ...","[(:, [0.26907914876937866, 0.530799925327301, ..."
3,"def _repr_html_(self):\n """"""\n J...","[('def', 'def', 'function_definition'), ('_rep...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 260, 1050, 62, 6494, 41052, 944, ...",221,"[('ers', 0.03083181567490101), ('(', 0.0166473...","[('anwhile', 1.03836617568492e-16), ('ousy', 4...","[('Ġ_', 0.001834857277572155), ('re', 0.001378...","{'type': 'module', 'children': [{'type': 'func...","[(list_comprehension, [0.13090811669826508]), ...","[(list_comprehension, [0.0015635205199941993])...","[(list_comprehension, [0.9856934547424316]), (..."
4,"def build_shape(relation, nodes, ways):\n ""...","[('def', 'def', 'function_definition'), ('buil...","[(4299, 'def', 'function_definition'), (1382, ...","[4299, 1382, 62, 43358, 7, 49501, 11, 13760, 1...",454,"[('ers', 0.03083181567490101), ('_', 0.2842166...","[('anwhile', 1.03836617568492e-16), ('buquerqu...","[('Ġbuild', 0.0005856865900568664), ('_', 0.28...","{'type': 'module', 'children': [{'type': 'func...","[(pass, [0.0024899975396692753]), (:, [0.02153...","[(pass, [0.0024899975396692753]), (:, [0.02153...","[(pass, [0.0024899975396692753]), (:, [0.02153..."


### Global Analysis (AST Elements)

In [12]:
def collect_global_std(std_field, dataframe):
    node_types_list = [[] for type in tokenizer.node_types]
    for tree in dataframe['binded_tree']:
        traverse_tree_and_collect_stds(tree, node_types_list, std_field)
    return node_types_list

In [13]:
concept_median_prob_list = collect_global_std('median_prob', df_actual_ntp)
concept_min_prob_list = collect_global_std('min_prob', df_actual_ntp)
concept_max_prob_list = collect_global_std('max_prob', df_actual_ntp)

In [14]:
global_concept_dataframe = pd.DataFrame([], columns=['ast_element', 'node_type' ,'concept_median_prob', 'concept_min_prob','concept_max_prob'])
for concept_idx in range(0,len(tokenizer.node_types)):
    if(len(concept_median_prob_list[concept_idx])>0):
        global_concept_dataframe.loc[len(global_concept_dataframe.index)] = [tokenizer.node_types[concept_idx],
                                                                             'parent' if tokenizer.node_types[concept_idx] in parent_node_types else 'leaf',
                                                                             concept_median_prob_list[concept_idx], 
                                                                             concept_min_prob_list[concept_idx], 
                                                                             concept_max_prob_list[concept_idx]]

In [15]:
global_concept_dataframe
## Anadir tipo de concepto columna 
# Quitar los vacios 

,ast_element,node_type,concept_median_prob,concept_min_prob,concept_max_prob
0,escape_sequence,leaf,"[0.6176046133041382, 0.4181915263470728, 0.481...","[0.3550148606300354, 0.0007045944803394377, 0....","[0.880194365978241, 0.8356784582138062, 0.9529..."
1,list_comprehension,parent,"[0.13090811669826508, 0.48584192991256714, 0.1...","[0.0015635205199941993, 0.0001130558448494412,...","[0.9856934547424316, 0.9513427019119263, 0.939..."
2,as_pattern_target,parent,"[0.22679175436496735, 0.013780202716588974, 0....","[0.0003977296582888812, 0.013780202716588974, ...","[0.790716826915741, 0.013780202716588974, 0.59..."
3,pass,leaf,"[0.0024899975396692753, 0.00965513288974762, 7...","[0.0024899975396692753, 0.00965513288974762, 7...","[0.0024899975396692753, 0.00965513288974762, 7..."
4,:,leaf,"[0.08046948164701462, 0.014872411265969276, 0....","[0.08046948164701462, 0.014872411265969276, 0....","[0.08046948164701462, 0.014872411265969276, 0...."
...,...,...,...,...,...
129,),leaf,"[0.08046948164701462, 0.014872411265969276, 0....","[0.08046948164701462, 0.014872411265969276, 0....","[0.08046948164701462, 0.014872411265969276, 0...."
130,block,parent,"[0.9981801509857178, 0.9997959733009338, 0.447...","[0.00014983986329752952, 0.3632635176181793, 0...","[0.9999983310699463, 0.9999979734420776, 0.502..."
131,-,leaf,"[0.013535365462303162, 0.5317811369895935, 0.9...","[0.013535365462303162, 0.5317811369895935, 0.9...","[0.013535365462303162, 0.5317811369895935, 0.9..."
132,!=,leaf,"[0.146612286567688, 0.17675115168094635, 0.056...","[0.146612286567688, 0.17675115168094635, 0.056...","[0.146612286567688, 0.17675115168094635, 0.056..."
